In [9]:
import pandas as pd
from zipfile import ZipFile
import numpy as np

# List of the sectors:
Sectors = ['Agriculture', 'Fishing', 'Mining and Quarrying']

# List with paths of each Eora zip file
ZIP_PATHS = np.array([r'. \Eora26_1990_bp.zip',
                      r'. \Eora26_1991_bp.zip',
                      r'. \Eora26_1992_bp.zip',
                      r'. \Eora26_1993_bp.zip',
                      r'. \Eora26_1994_bp.zip',
                      r'. \Eora26_1995_bp.zip',
                      r'. \Eora26_1996_bp.zip',
                      r'. \Eora26_1997_bp.zip',
                      r'. \Eora26_1998_bp.zip',
                      r'. \Eora26_1999_bp.zip',
                      r'. \Eora26_2000_bp.zip',
                      r'. \Eora26_2001_bp.zip',
                      r'. \Eora26_2002_bp.zip',
                      r'. \Eora26_2003_bp.zip',
                      r'. \Eora26_2004_bp.zip',
                      r'. \Eora26_2005_bp.zip',
                      r'. \Eora26_2006_bp.zip',
                      r'. \Eora26_2007_bp.zip',
                      r'. \Eora26_2008_bp.zip',
                      r'. \Eora26_2009_bp.zip',
                      r'. \Eora26_2010_bp.zip',
                      r'. \Eora26_2011_bp.zip',
                      r'. \Eora26_2012_bp.zip',
                      r'. \Eora26_2013_bp.zip',
                      r'. \Eora26_2014_bp.zip',
                      r'. \Eora26_2015_bp.zip',])

# Name of T tables in each zip file
T_Files = np.array(['Eora26_1990_bp_T.txt',
                    'Eora26_1991_bp_T.txt',
                    'Eora26_1992_bp_T.txt',
                    'Eora26_1993_bp_T.txt',
                    'Eora26_1994_bp_T.txt',
                    'Eora26_1995_bp_T.txt',
                    'Eora26_1996_bp_T.txt',
                    'Eora26_1997_bp_T.txt',
                    'Eora26_1998_bp_T.txt',
                    'Eora26_1999_bp_T.txt',
                    'Eora26_2000_bp_T.txt',
                    'Eora26_2001_bp_T.txt',
                    'Eora26_2002_bp_T.txt',
                    'Eora26_2003_bp_T.txt',
                    'Eora26_2004_bp_T.txt',
                    'Eora26_2005_bp_T.txt',
                    'Eora26_2006_bp_T.txt',
                    'Eora26_2007_bp_T.txt',
                    'Eora26_2008_bp_T.txt',
                    'Eora26_2009_bp_T.txt',
                    'Eora26_2010_bp_T.txt',
                    'Eora26_2011_bp_T.txt',
                    'Eora26_2012_bp_T.txt',
                    'Eora26_2013_bp_T.txt',
                    'Eora26_2014_bp_T.txt',
                    'Eora26_2015_bp_T.txt', ])

Table3_path = os.path.abspath('Data_for_regression_oct2023_TEST.xlsx')

df_Table3 = pd.read_excel(Table3_path, sheet_name='Table3')
df_Table3 = df_Table3[["Country",
                       "Year",
                       "Category",
                       "Category3",
                       "IsLowIncome",
                       "IsMiddleIncome",
                       "IsHighIncome",
                       "Total Production",
                       "BII",
                       "Delta BII ",
                       "DeltaBII_DECADAL",
                       "DeltaBII_ANNUAL",
                       "AVG(Country_land_area(sq.km))"]]

# Path with a Empty excel file named Trade_Eora.xlsx:
path = r'. \Trade_Eora.xlsx'

# book = load_workbook(path)
writer = pd.ExcelWriter(path, engine='openpyxl')

for sector in Sectors:
    T_Sector = pd.DataFrame()

    for zip_i in range(len(ZIP_PATHS)):

        zip_file = ZipFile(ZIP_PATHS[zip_i])  # open the zip file with data of one year
        T_year = pd.read_csv(zip_file.open(T_Files[zip_i]),  # read T table in that zip file
                             sep='\t',
                             header=None)
        df = T_year

        T_labels = pd.read_csv(zip_file.open('labels_T.txt'),  # open the labels for that T table in the same zip file
                               sep='\t',
                               header=None)
        # Select Countries abbreviation name and sector as index
        arrays = [T_labels.iloc[:, 0], T_labels.iloc[:, 1], T_labels.iloc[:, 2], T_labels.iloc[:, 3]]

        # Transform arrays to indexes
        MI = pd.MultiIndex.from_arrays(arrays, names=['Country', 'Country Abbr', 'Ind', 'Sector'])

        df = df.set_index(MI)  # Add indexes to rows of T table
        df.columns = pd.MultiIndex.from_arrays(arrays, names=['Country', 'Country Abr', 'Ind', 'Sector'])
        df = df.iloc[df.columns.get_level_values('Sector') == sector,
                     df.columns.get_level_values('Sector') == sector]

        for ind in ['Country', 'Sector', 'Ind']:
            df = df.droplevel(ind, axis=1)
            df = df.droplevel(ind, axis=0)
        df.index.name = 'Country'
        df.columns.name = ''
        Total_Exports = []
        for i in range(df.shape[0]):
            Total_Exports.append(df.iloc[i, :].sum() - df.iloc[i, i])
        Total_Imports = []
        for i in range(df.shape[1]):
            Total_Imports.append(df.iloc[:, i].sum() - df.iloc[i, i])
        Total_Exports_wic = []
        Total_Imports_wic = []
        for i in range(df.shape[0]):
            Total_Exports_wic.append(df.iloc[i, :].sum())
        for i in range(df.shape[1]):
            Total_Imports_wic.append(df.iloc[:, i].sum())
        df.insert(df.shape[1],
                  'Out Degree Centrality of the sector of country i, excluding internal consumption NOT NORMALIZED',
                  Total_Exports)
        df.insert(df.shape[1],
                  'In Degree Centrality of the sector of country i, excluding internal consumption NOT NORMALIZED',
                  Total_Imports)
        df.insert(df.shape[1], 'total exports of the sector, all countries, excluding internal consumption ( Σi  )',
                  np.array(Total_Exports).sum())
        df.insert(df.shape[1],
                  'Out Degree Centrality of the sector of country i, excluding internal consumption NORMALIZED (by Σi)',
                  np.divide(Total_Exports, np.array(Total_Exports).sum()))
        df.insert(df.shape[1],
                  'In degree centrality of the sector of country i, excluding internal consumption NORMALIZED (by Σi)',
                  np.divide(Total_Imports, np.array(Total_Exports).sum()))
        df.insert(df.shape[1],
                  'Out Degree Centrality of the sector in country i, INCLUDING internal consumption NOT NORMALIZED',
                  Total_Exports_wic)
        df.insert(df.shape[1],
                  'In Degree Centrality of the sector in country i, INCLUDING internal consumption NOT NORMALIZED',
                  Total_Imports_wic)
        EDSij = pd.DataFrame()
        for i in range(189):
            EDSij = pd.concat([EDSij,
                               df.iloc[:, i].div(df['Out Degree Centrality of the sector of country i, excluding internal consumption NORMALIZED (by Σi)'])],
                              axis=1)
        #   EDSij.iloc[i,i] = np.nan
        # EDSij.columns[i]=df.iloc[:,i].name+' EDS'
        EDSij.columns = EDSij.index + ' Export Destination Share'
        df = pd.concat([df, EDSij], axis=1)
        EDSijnnan = EDSij.copy()
        for i in range(189):
            EDSijnnan.iloc[i, i] = np.nan
        SUMEDSi = []
        for i in range(189):
            SUMEDSi.append(EDSijnnan.iloc[i].sum())
        SUMEDSi = pd.DataFrame(SUMEDSi)
        SUMEDSi.index = df.index
        SUMEDSi.columns = ['SUM EDS for country i, without internal consumption']
        df = pd.concat([df, SUMEDSi], axis=1)
        IPF = pd.DataFrame(
            df['In degree centrality of the sector of country i, excluding internal consumption NORMALIZED (by Σi)'] *
            df['SUM EDS for country i, without internal consumption'], index=df.index, columns=['Importer Power Face'])
        df = pd.concat([df, IPF], axis=1)
        # df.insert(df.shape[1], 'Sum of EDS without internal consume', SUMEDSi)
        #    df.insert(0, 'Year', 1990 + zip_i)  # Add the year of the zip file as column
        df = pd.concat([pd.DataFrame([1990 + zip_i] * len(df.index), index=df.index, columns=['Year']), df], axis=1)
        df.index.name = 'Country'
        df = df.reset_index()
        df = df.merge(df_Table3, on=['Country', 'Year'])
        T_Sector = pd.concat([T_Sector, df])

    T_Sector.to_excel(writer, sheet_name=sector)

#    Sheets.append(T_Sector)

# print(Sheets)
writer.close()

FileNotFoundError: [Errno 2] No such file or directory: '. \\Eora26_1990_bp.zip'

In [11]:
import os
import pandas as pd
from zipfile import ZipFile
import numpy as np

# List of the sectors:
Sectors = ['Agriculture', 'Fishing', 'Mining and Quarrying']

# List with paths of each Eora zip file (corrected)
ZIP_PATHS = np.array([r'Eora26_1990_bp.zip',
                      r'Eora26_1991_bp.zip',
                      r'Eora26_1992_bp.zip',
                      r'Eora26_1993_bp.zip',
                      r'Eora26_1994_bp.zip',
                      r'Eora26_1995_bp.zip',
                      r'Eora26_1996_bp.zip',
                      r'Eora26_1997_bp.zip',
                      r'Eora26_1998_bp.zip',
                      r'Eora26_1999_bp.zip',
                      r'Eora26_2000_bp.zip',
                      r'Eora26_2001_bp.zip',
                      r'Eora26_2002_bp.zip',
                      r'Eora26_2003_bp.zip',
                      r'Eora26_2004_bp.zip',
                      r'Eora26_2005_bp.zip',
                      r'Eora26_2006_bp.zip',
                      r'Eora26_2007_bp.zip',
                      r'Eora26_2008_bp.zip',
                      r'Eora26_2009_bp.zip',
                      r'Eora26_2010_bp.zip',
                      r'Eora26_2011_bp.zip',
                      r'Eora26_2012_bp.zip',
                      r'Eora26_2013_bp.zip',
                      r'Eora26_2014_bp.zip',
                      r'Eora26_2015_bp.zip'])

# Name of T tables in each zip file
T_Files = np.array(['Eora26_1990_bp_T.txt',
                    'Eora26_1991_bp_T.txt',
                    'Eora26_1992_bp_T.txt',
                    'Eora26_1993_bp_T.txt',
                    'Eora26_1994_bp_T.txt',
                    'Eora26_1995_bp_T.txt',
                    'Eora26_1996_bp_T.txt',
                    'Eora26_1997_bp_T.txt',
                    'Eora26_1998_bp_T.txt',
                    'Eora26_1999_bp_T.txt',
                    'Eora26_2000_bp_T.txt',
                    'Eora26_2001_bp_T.txt',
                    'Eora26_2002_bp_T.txt',
                    'Eora26_2003_bp_T.txt',
                    'Eora26_2004_bp_T.txt',
                    'Eora26_2005_bp_T.txt',
                    'Eora26_2006_bp_T.txt',
                    'Eora26_2007_bp_T.txt',
                    'Eora26_2008_bp_T.txt',
                    'Eora26_2009_bp_T.txt',
                    'Eora26_2010_bp_T.txt',
                    'Eora26_2011_bp_T.txt',
                    'Eora26_2012_bp_T.txt',
                    'Eora26_2013_bp_T.txt',
                    'Eora26_2014_bp_T.txt',
                    'Eora26_2015_bp_T.txt'])

# Absolute path to the Excel file (corrected)
Table3_path = os.path.abspath('Data_for_regression_oct2023_TEST.xlsx')
print("Current Working Directory:", os.getcwd())
print("Files in Current Directory:", os.listdir(os.getcwd()))
print("Absolute Path to Table3:", Table3_path)
print("Does Table3 file exist?", os.path.exists(Table3_path))

# Attempt to read the Excel file
try:
    df_Table3 = pd.read_excel(Table3_path, sheet_name='Table3')
    df_Table3 = df_Table3[["Country",
                           "Year",
                           "Category",
                           "Category3",
                           "IsLowIncome",
                           "IsMiddleIncome",
                           "IsHighIncome",
                           "Total Production",
                           "BII",
                           "Delta BII ",
                           "DeltaBII_DECADAL",
                           "DeltaBII_ANNUAL",
                           "AVG(Country_land_area(sq.km))"]]
    print("Successfully read Table3")
except FileNotFoundError as e:
    print("FileNotFoundError:", e)
except Exception as e:
    print("Error:", e)

# Path with an empty Excel file named Trade_Eora.xlsx:
path = os.path.abspath('Trade_Eora.xlsx')
print("Writing to Trade_Eora at:", path)

# List contents of each ZIP file
for zip_i, zip_path in enumerate(ZIP_PATHS):
    print(f"\nListing contents of {zip_path}:")
    try:
        with ZipFile(zip_path, 'r') as zip_file:
            zip_contents = zip_file.namelist()
            print(zip_contents)
            if T_Files[zip_i] not in zip_contents:
                print(f"Warning: {T_Files[zip_i]} not found in {zip_path}")
    except FileNotFoundError as e:
        print("FileNotFoundError:", e)
    except Exception as e:
        print("Error:", e)

# Continue with the rest of your code if no errors
# book = load_workbook(path)
writer = pd.ExcelWriter(path, engine='openpyxl')

for sector in Sectors:
    T_Sector = pd.DataFrame()

    for zip_i in range(len(ZIP_PATHS)):

        zip_file = ZipFile(ZIP_PATHS[zip_i])  # open the zip file with data of one year
        T_year = pd.read_csv(zip_file.open(T_Files[zip_i]),  # read T table in that zip file
                             sep='\t',
                             header=None)
        df = T_year

        T_labels = pd.read_csv(zip_file.open('labels_T.txt'),  # open the labels for that T table in the same zip file
                               sep='\t',
                               header=None)
        # Select Countries abbreviation name and sector as index
        arrays = [T_labels.iloc[:, 0], T_labels.iloc[:, 1], T_labels.iloc[:, 2], T_labels.iloc[:, 3]]

        # Transform arrays to indexes
        MI = pd.MultiIndex.from_arrays(arrays, names=['Country', 'Country Abbr', 'Ind', 'Sector'])

        df = df.set_index(MI)  # Add indexes to rows of T table
        df.columns = pd.MultiIndex.from_arrays(arrays, names=['Country', 'Country Abr', 'Ind', 'Sector'])
        df = df.iloc[df.columns.get_level_values('Sector') == sector,
                     df.columns.get_level_values('Sector') == sector]

        for ind in ['Country', 'Sector', 'Ind']:
            df = df.droplevel(ind, axis=1)
            df = df.droplevel(ind, axis=0)
        df.index.name = 'Country'
        df.columns.name = ''
        Total_Exports = []
        for i in range(df.shape[0]):
            Total_Exports.append(df.iloc[i, :].sum() - df.iloc[i, i])
        Total_Imports = []
        for i in range(df.shape[1]):
            Total_Imports.append(df.iloc[:, i].sum() - df.iloc[i, i])
        Total_Exports_wic = []
        Total_Imports_wic = []
        for i in range(df.shape[0]):
            Total_Exports_wic.append(df.iloc[i, :].sum())
        for i in range(df.shape[1]):
            Total_Imports_wic.append(df.iloc[:, i].sum())
        df.insert(df.shape[1],
                  'Out Degree Centrality of the sector of country i, excluding internal consumption NOT NORMALIZED',
                  Total_Exports)
        df.insert(df.shape[1],
                  'In Degree Centrality of the sector of country i, excluding internal consumption NOT NORMALIZED',
                  Total_Imports)
        df.insert(df.shape[1], 'total exports of the sector, all countries, excluding internal consumption ( Σi  )',
                  np.array(Total_Exports).sum())
        df.insert(df.shape[1],
                  'Out Degree Centrality of the sector of country i, excluding internal consumption NORMALIZED (by Σi)',
                  np.divide(Total_Exports, np.array(Total_Exports).sum()))
        df.insert(df.shape[1],
                  'In degree centrality of the sector of country i, excluding internal consumption NORMALIZED (by Σi)',
                  np.divide(Total_Imports, np.array(Total_Exports).sum()))
        df.insert(df.shape[1],
                  'Out Degree Centrality of the sector in country i, INCLUDING internal consumption NOT NORMALIZED',
                  Total_Exports_wic)
        df.insert(df.shape[1],
                  'In Degree Centrality of the sector in country i, INCLUDING internal consumption NOT NORMALIZED',
                  Total_Imports_wic)
        EDSij = pd.DataFrame()
        for i in range(189):
            EDSij = pd.concat([EDSij,
                               df.iloc[:, i].div(df['Out Degree Centrality of the sector of country i, excluding internal consumption NORMALIZED (by Σi)'])],
                              axis=1)
        #   EDSij.iloc[i,i] = np.nan
        # EDSij.columns[i]=df.iloc[:,i].name+' EDS'
        EDSij.columns = EDSij.index + ' Export Destination Share'
        df = pd.concat([df, EDSij], axis=1)
        EDSijnnan = EDSij.copy()
        for i in range(189):
            EDSijnnan.iloc[i, i] = np.nan
        SUMEDSi = []
        for i in range(189):
            SUMEDSi.append(EDSijnnan.iloc[i].sum())
        SUMEDSi = pd.DataFrame(SUMEDSi)
        SUMEDSi.index = df.index
        SUMEDSi.columns = ['SUM EDS for country i, without internal consumption']
        df = pd.concat([df, SUMEDSi], axis=1)
        IPF = pd.DataFrame(
            df['In degree centrality of the sector of country i, excluding internal consumption NORMALIZED (by Σi)'] *
            df['SUM EDS for country i, without internal consumption'], index=df.index, columns=['Importer Power Face'])
        df = pd.concat([df, IPF], axis=1)
        # df.insert(df.shape[1], 'Sum of EDS without internal consume', SUMEDSi)
        #    df.insert(0, 'Year', 1990 + zip_i)  # Add the year of the zip file as column
        df = pd.concat([pd.DataFrame([1990 + zip_i] * len(df.index), index=df.index, columns=['Year']), df], axis=1)
        df.index.name = 'Country'
        df = df.reset_index()
        df = df.merge(df_Table3, on=['Country', 'Year'])
        T_Sector = pd.concat([T_Sector, df])

    T_Sector.to_excel(writer, sheet_name=sector)

#    Sheets.append(T_Sector)

# print(Sheets)
writer.close()

Current Working Directory: /Users/LittleStupidCuteBear/Desktop/Research/20240709 new IPF with rectangular matrix
Files in Current Directory: ['Eora26_1990_bp', 'Eora Generator_new.ipynb', 'labels_T.txt', 'Eora26_2002_bp.zip', 'Eora26_2010_bp.zip', 'Eora26_1996_bp.zip', '.DS_Store', 'Eora26_1998_bp.zip', 'Eora26_1994_bp.zip', 'Trade_Eora.xlsx', 'Eora26_2012_bp.zip', 'Eora26_2000_bp.zip', 'Eora26_1990_bp.zip', 'Eora26_2004_bp.zip', 'Trade_Eora_Generator.py', 'Eora26_2008_bp.zip', 'IPF_calculation_new.ipynb', 'Eora26_2014_bp.zip', 'Eora26_2006_bp.zip', '~$IPF_Mining and Quarrying.xlsx', 'Eora26_1992_bp.zip', 'IPF_Mining and Quarrying.xlsx', 'Eora26_1997_bp.zip', 'Eora26_2003_bp.zip', '. \\Trade_Eora.xlsx', 'Eora26_2011_bp.zip', 'Eora26_2013_bp.zip', 'Eora26_2001_bp.zip', 'Eora26_1999_bp.zip', 'Eora26_1995_bp.zip', '~$Trade_Eora.xlsx', '.ipynb_checkpoints', 'Eora26_2005_bp.zip', 'Eora26_2009_bp.zip', 'Data_for_regression_oct2023_TEST.xlsx', 'Eora26_1991_bp.zip', 'Eora26_1993_bp.zip', 'Eora

KeyError: "There is no item named 'Eora26_1990_bp_T.txt' in the archive"

In [12]:
import os
from zipfile import ZipFile

# List with paths of each Eora zip file
ZIP_PATHS = [
    'Eora26_1990_bp.zip', 'Eora26_1991_bp.zip', 'Eora26_1992_bp.zip', 'Eora26_1993_bp.zip', 'Eora26_1994_bp.zip',
    'Eora26_1995_bp.zip', 'Eora26_1996_bp.zip', 'Eora26_1997_bp.zip', 'Eora26_1998_bp.zip', 'Eora26_1999_bp.zip',
    'Eora26_2000_bp.zip', 'Eora26_2001_bp.zip', 'Eora26_2002_bp.zip', 'Eora26_2003_bp.zip', 'Eora26_2004_bp.zip',
    'Eora26_2005_bp.zip', 'Eora26_2006_bp.zip', 'Eora26_2007_bp.zip', 'Eora26_2008_bp.zip', 'Eora26_2009_bp.zip',
    'Eora26_2010_bp.zip', 'Eora26_2011_bp.zip', 'Eora26_2012_bp.zip', 'Eora26_2013_bp.zip', 'Eora26_2014_bp.zip',
    'Eora26_2015_bp.zip'
]

# Name of T tables in each zip file
T_Files = [
    'Eora26_1990_bp_T.txt', 'Eora26_1991_bp_T.txt', 'Eora26_1992_bp_T.txt', 'Eora26_1993_bp_T.txt', 'Eora26_1994_bp_T.txt',
    'Eora26_1995_bp_T.txt', 'Eora26_1996_bp_T.txt', 'Eora26_1997_bp_T.txt', 'Eora26_1998_bp_T.txt', 'Eora26_1999_bp_T.txt',
    'Eora26_2000_bp_T.txt', 'Eora26_2001_bp_T.txt', 'Eora26_2002_bp_T.txt', 'Eora26_2003_bp_T.txt', 'Eora26_2004_bp_T.txt',
    'Eora26_2005_bp_T.txt', 'Eora26_2006_bp_T.txt', 'Eora26_2007_bp_T.txt', 'Eora26_2008_bp_T.txt', 'Eora26_2009_bp_T.txt',
    'Eora26_2010_bp_T.txt', 'Eora26_2011_bp_T.txt', 'Eora26_2012_bp_T.txt', 'Eora26_2013_bp_T.txt', 'Eora26_2014_bp_T.txt',
    'Eora26_2015_bp_T.txt'
]

# List contents of each ZIP file
for zip_i, zip_path in enumerate(ZIP_PATHS):
    print(f"\nListing contents of {zip_path}:")
    try:
        with ZipFile(zip_path, 'r') as zip_file:
            zip_contents = zip_file.namelist()
            print(zip_contents)
            if T_Files[zip_i] not in zip_contents:
                print(f"Warning: {T_Files[zip_i]} not found in {zip_path}")
            else:
                print(f"File {T_Files[zip_i]} found in {zip_path}")
    except FileNotFoundError as e:
        print("FileNotFoundError:", e)
    except Exception as e:
        print("Error:", e)


Listing contents of Eora26_1990_bp.zip:
['Eora26_1990_bp/', '__MACOSX/._Eora26_1990_bp', 'Eora26_1990_bp/labels_T.txt', '__MACOSX/Eora26_1990_bp/._labels_T.txt', 'Eora26_1990_bp/labels_VA.txt', '__MACOSX/Eora26_1990_bp/._labels_VA.txt', 'Eora26_1990_bp/Eora26_1990_bp_T.txt', '__MACOSX/Eora26_1990_bp/._Eora26_1990_bp_T.txt', 'Eora26_1990_bp/Eora26_1990_bp_VA.txt', '__MACOSX/Eora26_1990_bp/._Eora26_1990_bp_VA.txt', 'Eora26_1990_bp/Eora26_1990_bp_Q.txt', '__MACOSX/Eora26_1990_bp/._Eora26_1990_bp_Q.txt', 'Eora26_1990_bp/labels_T.xlsx', '__MACOSX/Eora26_1990_bp/._labels_T.xlsx', 'Eora26_1990_bp/Eora26_1990_bp_T.xlsx', '__MACOSX/Eora26_1990_bp/._Eora26_1990_bp_T.xlsx', 'Eora26_1990_bp/labels_FD.txt', '__MACOSX/Eora26_1990_bp/._labels_FD.txt', 'Eora26_1990_bp/Eora26_1990_bp_QY.txt', '__MACOSX/Eora26_1990_bp/._Eora26_1990_bp_QY.txt', 'Eora26_1990_bp/Eora26_1990_bp_FD.txt', '__MACOSX/Eora26_1990_bp/._Eora26_1990_bp_FD.txt', 'Eora26_1990_bp/import/', '__MACOSX/Eora26_1990_bp/._import', 'Eora26_

In [5]:
df

,Var1
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
4129,NaN
4130,NaN
4131,NaN
4132,NaN


In [14]:
import zipfile

def list_zip_contents(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_names = zip_ref.namelist()
        for file_name in file_names:
            print(file_name)

# Example usage
list_zip_contents('./Eora26_1990_bp.zip')

Eora26_1990_bp/
__MACOSX/._Eora26_1990_bp
Eora26_1990_bp/labels_T.txt
__MACOSX/Eora26_1990_bp/._labels_T.txt
Eora26_1990_bp/labels_VA.txt
__MACOSX/Eora26_1990_bp/._labels_VA.txt
Eora26_1990_bp/Eora26_1990_bp_T.txt
__MACOSX/Eora26_1990_bp/._Eora26_1990_bp_T.txt
Eora26_1990_bp/Eora26_1990_bp_VA.txt
__MACOSX/Eora26_1990_bp/._Eora26_1990_bp_VA.txt
Eora26_1990_bp/Eora26_1990_bp_Q.txt
__MACOSX/Eora26_1990_bp/._Eora26_1990_bp_Q.txt
Eora26_1990_bp/labels_T.xlsx
__MACOSX/Eora26_1990_bp/._labels_T.xlsx
Eora26_1990_bp/Eora26_1990_bp_T.xlsx
__MACOSX/Eora26_1990_bp/._Eora26_1990_bp_T.xlsx
Eora26_1990_bp/labels_FD.txt
__MACOSX/Eora26_1990_bp/._labels_FD.txt
Eora26_1990_bp/Eora26_1990_bp_QY.txt
__MACOSX/Eora26_1990_bp/._Eora26_1990_bp_QY.txt
Eora26_1990_bp/Eora26_1990_bp_FD.txt
__MACOSX/Eora26_1990_bp/._Eora26_1990_bp_FD.txt
Eora26_1990_bp/import/
__MACOSX/Eora26_1990_bp/._import
Eora26_1990_bp/index_readme.txt
__MACOSX/Eora26_1990_bp/._index_readme.txt
Eora26_1990_bp/import/miranda/
__MACOSX/Eora26_